# Assignment 7

## Submit as an HTML file

### Print your name below

In [31]:
print("Meredith Teague")

Meredith Teague


### Import the "pandas" "numpy" and "statsmodels.formula.api" libraries

In [32]:
# Write your answer here:
import pandas as pd 
import numpy as np 
import statsmodels.formula.api as smf




#### In the code chunk below read the CSV file named `results.csv` in the `data` <br> folder and print the first 5 rows of the dataset. Browse the dataset.

In [33]:
data = pd.read_csv("data/results.csv")
print(data.head)

<bound method NDFrame.head of        resultId  raceId  driverId  constructorId number  grid position  \
0             1      18         1              1     22     1        1   
1             2      18         2              2      3     5        2   
2             3      18         3              3      7     7        3   
3             4      18         4              4      5    11        4   
4             5      18         5              1     23     3        5   
...         ...     ...       ...            ...    ...   ...      ...   
25835     25841    1096       854            210     47    12       16   
25836     25842    1096       825            210     20    16       17   
25837     25843    1096         1            131     44     5       18   
25838     25844    1096       849              3      6    20       19   
25839     25845    1096         4            214     14    10       \N   

      positionText  positionOrder  points  laps         time milliseconds  \
0   

### (a)  Check Column Types and Data Cleaning

- Use the function .dtypes to get the column types
- Identify which columns have data types that might need conversion
- The 'milliseconds' column contains string values that should be numeric. Create a new column called 'race_time_ms' that:
    - Converts the column to a numeric data type
    - Replaces any non-numeric values with NaN

In [49]:
# Write your answer here
data.dtypes
data["race_time_ms"] = pd.to_numeric(data["milliseconds"].replace('\\N', np.nan))




### (b) Create Categorical Variables

- Create a new column called 'finish_category' that categorizes the race finish positions as follows:
    - Positions 1-3: 'Podium'
    - Positions 4-10: 'Points'
    - Positions 11-20: 'Midfield'
    - Positions >20: 'Backmarker'

Hint: Use the pd.cut() function

In [50]:
# Write your answer here

bins = [0, 3, 10, 20, 1000000000]  
labels = ['Podium', 'Points', 'Midfield', 'Backmarker']

data['finish_category'] = pd.cut(data['positionOrder'], bins=bins, labels=labels)
display(data)




,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,finish_category,race_duration_minutes,race_time_ms
0,1,18,1,1,22,1,1,1,1,10.0,...,1:34:50.616,5690616,39,2,1:27.452,218.300,1,Podium,94.843600,5690616.0
1,2,18,2,2,3,5,2,2,2,8.0,...,+5.478,5696094,41,3,1:27.739,217.586,1,Podium,94.934900,5696094.0
2,3,18,3,3,7,7,3,3,3,6.0,...,+8.163,5698779,41,5,1:28.090,216.719,1,Podium,94.979650,5698779.0
3,4,18,4,4,5,11,4,4,4,5.0,...,+17.181,5707797,58,7,1:28.603,215.464,1,Points,95.129950,5707797.0
4,5,18,5,1,23,3,5,5,5,4.0,...,+18.014,5708630,43,1,1:27.418,218.385,1,Points,95.143833,5708630.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25835,25841,1096,854,210,47,12,16,16,16,0.0,...,\N,NaN,39,12,1:29.833,211.632,11,Midfield,NaN,NaN
25836,25842,1096,825,210,20,16,17,17,17,0.0,...,\N,NaN,40,20,1:31.158,208.556,11,Midfield,NaN,NaN
25837,25843,1096,1,131,44,5,18,18,18,0.0,...,\N,NaN,42,11,1:29.788,211.738,9,Midfield,NaN,NaN
25838,25844,1096,849,3,6,20,19,19,19,0.0,...,\N,NaN,45,14,1:30.309,210.517,130,Midfield,NaN,NaN


### (c) Calculate Race Duration
- For rows where 'milliseconds' is available, create a new column <br>
'race_duration_minutes' that converts milliseconds to minutes by dividing <br>
by (1000*60).
- Display the average race duration by 'constructorId' for the top 5 <br>
constructors with the shortest average race times

In [51]:
# Write your answer here
data['race_duration_minutes'] = data["race_time_ms"] / (1000 * 60)


avgraceduration = data.groupby('constructorId')['race_duration_minutes'].mean()

top5shortest = avgraceduration.nsmallest(5)
print(top5shortest)



constructorId
35    76.710777
29    77.604125
41    87.046767
16    89.428828
53    89.658852
Name: race_duration_minutes, dtype: float64


### (d) Driver Performance Analysis

- Calculate the following statistics for each driver, grouped by 'driverId':
    - Average finishing position
    - Total points
    - Number of races completed
    - Best finishing position

- Sort the results by total points in descending order
- Display the top 10 drivers based on total points

In [39]:
# Write your answer here
driverStats = data.groupby('driverId').agg(
    avgfinishposition=('positionOrder', 'mean'),
    totalpoints=('points', 'sum'),
    racescompleted=('raceId', 'count'),
    bestfinish=('positionOrder', 'min')
)

driverstatssorted = driverStats.sort_values(by='totalpoints', ascending=False)

print(driverstatssorted.head(10))


          avgfinishposition  totalpoints  racescompleted  bestfinish
driverId                                                            
1                  4.787097       4396.5             310           1
20                 7.093333       3098.0             300           1
4                  8.494413       2061.0             358           1
830                6.533742       1983.5             163           1
8                  8.491477       1873.0             352           1
822                7.601990       1778.0             201           1
3                  8.252427       1594.5             206           1
30                 6.879870       1566.0             308           1
817                9.883621       1307.0             232           1
18                 9.695793       1235.0             309           1


### (e) Linear Regression
Create a linear regression model that predicts 'points' based on 'grid' (starting position) and 'laps' completed <br>
Use the following steps:

- Clean the data to remove any non-numeric values and missing values
- Create the regression formula using smf.ols 
- Display the summary of the regression model using model.summary()

What is the predicted points for a driver starting in position 3 and completing 55 laps?

Hint: Use ```.dropna()''' to remove missing values from the points, grid, and laps <br>
variables.

In [29]:
# Write your answer here
data_clean = data[['points', 'grid', 'laps']].dropna()

model = smf.ols('points ~ grid + laps', data=data_clean).fit()

print(model.summary())

predicted_points = model.predict(pd.DataFrame({'grid': [3], 'laps': [55]}))
print(predicted_points)



                            OLS Regression Results                            
Dep. Variable:                 points   R-squared:                       0.215
Model:                            OLS   Adj. R-squared:                  0.215
Method:                 Least Squares   F-statistic:                     3530.
Date:                Mon, 24 Mar 2025   Prob (F-statistic):               0.00
Time:                        21:07:13   Log-Likelihood:                -70440.
No. Observations:               25840   AIC:                         1.409e+05
Df Residuals:                   25837   BIC:                         1.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.5841      0.054     48.267      0.0